Задача: \
Построить модель машинного обучения для решения задачи регрессии: необходимо спрогнозировать прогресс заболевания диабетом через год после исходного уровня.
- В документации библиотеки scikit learn найдите, как загрузить датасет для построения модели прогноза прогрессирования заболевания через год после исходного уровня (load_diabetes из sklearn.datasets).
- Выведите содержимое поля DESCR, которое вернет функция, загружающая датасет, чтобы изучить содержимое датасета.
- Подготовьте данные к обучению моделей: отделите целевой признак от датасета, разделите датасет на тренировочную и валидационную части.
- Обучите решающее дерево и оцените адекватность обученной модели. Для этого достаточно сравнить метрики качества с метриками, получаемыми от простой модели. Например, если бы мы подавали на вход среднее значение целевой переменной на обучающей части выборки.
- Обучите линейную регрессию и оцените ее адекватность.
- Выберите лучшую модель и обоснуйте свой выбор.

#### Загрузка данных

In [1]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import linear_model

#### Описание данных

In [2]:
print(load_diabetes().DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [3]:
df = load_diabetes(as_frame=True, scaled=False).frame

In [4]:
df.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [5]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,59.0,2.0,32.1,101.0,157.0,93.2,38.0,4.0,4.8598,87.0,151.0
1,48.0,1.0,21.6,87.0,183.0,103.2,70.0,3.0,3.8918,69.0,75.0
2,72.0,2.0,30.5,93.0,156.0,93.6,41.0,4.0,4.6728,85.0,141.0
3,24.0,1.0,25.3,84.0,198.0,131.4,40.0,5.0,4.8903,89.0,206.0
4,50.0,1.0,23.0,101.0,192.0,125.4,52.0,4.0,4.2905,80.0,135.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


#### Обучение  модели

In [7]:
# выделение таргета и признаков в отдельные переменные
features = df.drop(["target"], axis=1)
target = df["target"]

In [8]:
# разбивка датасета 80/20
features_train, features_val, target_train, target_val = train_test_split(
    features, target, test_size=.2, random_state=123
)

In [9]:
# генерация тестовых данных по среднему
mean_val = pd.Series(target_train.mean(), [i for i in range(target_val.shape[0])])
mean_val

0     152.427762
1     152.427762
2     152.427762
3     152.427762
4     152.427762
         ...    
84    152.427762
85    152.427762
86    152.427762
87    152.427762
88    152.427762
Length: 89, dtype: float64

In [10]:
def show_metrics(model, metrica):
    """
    Function for show metrics from input model
    """
    model.fit(features_train, target_train)
    predictions = model.predict(features_val)
    print(f"Model {model}")
    print(
        f"{(metrica.__name__).title()} validation data is {round(metrica(target_val, predictions), 6)}"
    )
    print(
        f"{(metrica.__name__).title()} mean data is {round(metrica(target_val, mean_val), 6)}"
    )
    print("-------------")

*MAE*

In [11]:
model_tree = tree.DecisionTreeRegressor(
    criterion="absolute_error", max_depth=7, min_samples_leaf=3, ccp_alpha=0.5
)
show_metrics(model_tree, mean_absolute_error)
model_lr = linear_model.LinearRegression()
show_metrics(model_lr, mean_absolute_error)

Model DecisionTreeRegressor(ccp_alpha=0.5, criterion='absolute_error', max_depth=7,
                      min_samples_leaf=3)
Mean_Absolute_Error validation data is 50.966292
Mean_Absolute_Error mean data is 68.698189
-------------
Model LinearRegression()
Mean_Absolute_Error validation data is 42.87613
Mean_Absolute_Error mean data is 68.698189
-------------


*squared_error*

In [12]:
model_tree = tree.DecisionTreeRegressor(
    criterion="squared_error", max_depth=3, min_samples_leaf=3
)
show_metrics(model_tree, r2_score)
model_lr = linear_model.LinearRegression()
show_metrics(model_lr, r2_score)

Model DecisionTreeRegressor(max_depth=3, min_samples_leaf=3)
R2_Score validation data is 0.454764
R2_Score mean data is -0.000339
-------------
Model LinearRegression()
R2_Score validation data is 0.56759
R2_Score mean data is -0.000339
-------------


*MSE*

In [13]:
model_tree = tree.DecisionTreeRegressor(criterion="friedman_mse")
show_metrics(model_tree, mean_squared_error)
model_lr = linear_model.LinearRegression()
show_metrics(model_lr, mean_squared_error)

Model DecisionTreeRegressor(criterion='friedman_mse')
Mean_Squared_Error validation data is 5369.067416
Mean_Squared_Error mean data is 6302.303297
-------------
Model LinearRegression()
Mean_Squared_Error validation data is 2724.25808
Mean_Squared_Error mean data is 6302.303297
-------------


*Вывод*:\
Лучше показала себя модель LinearRegression:
- Mean_Absolute_Error 42.87613
- Mean_Squared_Error 2724.25808
- R2_Score 0.56759